## On this notebook we extract Censo 2010 individual data from their files.


In [72]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


/tmp/ipykernel_184550/3391910190.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [73]:
import os

if not os.path.exists('./../../samplerCensoARG/'):
    !git clone https://github.com/matuteiglesias/samplerCensoARG.git ./../../

In [74]:
# !python ./../../samplerCensoARG/samplear.py -dbp '/media/miglesia/Elements/suite/ext_CPV2010_basico_radio_pub' -f 0.01 -y 2005 2006 -n ARG

In [75]:
import pandas as pd
# pd.read_csv('/home/miglesia/repositories/indice-pobreza-ExactasUBA/notebooks/../data/info/AGLO_rk/')

## Cargar info accesoria (Regiones, aglomerados, etc)

In [76]:
radio_ref = pd.read_csv('./../data/info/radio_ref.csv')

radio_AGLO = pd.read_csv('https://raw.githubusercontent.com/matuteiglesias/Aglomerados-EPH-INDEC/main/result/radios_aglo_EPH.csv')
radio_AGLO['radio'] = radio_AGLO.COD_2010.str.replace('XX', '99').astype(int)
radio_AGLO['AGLOMERADO'] = radio_AGLO.eph_codagl
radio_AGLO['NOMAGLO'] = radio_AGLO.eph_aglome

radio_ref = radio_ref.drop(['AGLOMERADO'], axis = 1).merge(radio_AGLO[['radio','AGLOMERADO', 'NOMAGLO']], how = 'left')
radio_ref['AGLOMERADO'] = radio_ref['AGLOMERADO'].fillna(0).astype(int)

# radio_ref[['PROV','NOMPROV','DPTO', 'NOMDPTO']].drop_duplicates().to_csv('./../data/DPTO_PROV.csv', index = False)
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
radio_ref = radio_ref.merge(dpto_region)

In [77]:

AGLO_rk = pd.read_csv('./../../encuestador-de-hogares/data/info/AGLO_rk')
rk_table = AGLO_rk.set_index(['ANO4', 'AGLOMERADO']).unstack()
AGLO_rk_filled = rk_table.fillna(rk_table.mean()).stack().reset_index()
AGLO_rk = AGLO_rk_filled

Reg_rk = pd.read_csv('./../../encuestador-de-hogares/data/info/Reg_rk')


## Loop sampleo de censo

In [78]:
import numpy as np

In [79]:
pd.read_csv('./../../samplerCensoARG/data/censo_samples/table_f'+str(frac)+'_'+yr+'_ARG.csv', nrows = 5).columns

Index(['VIVIENDA_REF_ID', 'RADIO_REF_ID', 'TIPVV', 'V01', 'URP', 'DPTO',
       'PROV', 'AGLOMERADO', 'HOGAR_REF_ID', 'H05', 'H06', 'H07', 'H08', 'H09',
       'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'PROP', 'TOTPERS',
       'PERSONA_REF_ID', 'P01', 'P02', 'P03', 'P05', 'P06', 'P07', 'P12',
       'P08', 'P09', 'P10', 'CONDACT'],
      dtype='object')

In [80]:
frac = 0.01 ## Frac needs to be the fraction used in the sampling (eg. -f 0.01 needs frac = 0.01)
startyr = 2015
endyr = 2024

for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    table = pd.read_csv('./../../samplerCensoARG/data/censo_samples/table_f'+str(frac)+'_'+yr+'_ARG.csv')
    table['ANO4'] = int(yr)
    
    # Adaptamos las categorias de respuestas para que iguales las de la EPH
    ## VIVIENDA
    table['V01'] = table['V01'].map({1:1, 2:6, 3:6, 4:2, 5:3, 6:4, 7:5, 8:6})
    ## HOGAR
    table['H06'] = table['H06'].map({1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:9})
    table['H09'] = table['H09'].map({1:1, 2:2, 3:3, 4:4, 5:4, 6:4})
    table['H16'] = table['H16'].clip(0, 9)
    table['H14'] = table['H14'].map({1:1, 2:4, 3:2, 4:2, 5:4, 6:3, 7:4, 8:9})
    table['H13'] = table['H13'].map({1:1, 2:2, 4:0})
    table['IX_TOT'] = table['TOTPERS']
    # PERSONA
    table['P07'] = table['P07'].map({1:1, 2:2, 0:2})

    ## Agregar Region
    table = table.merge(dpto_region[['DPTO', 'Region']])

    ## ID DE PERSONAS SIMULADAS:
    # Semilla ids
    file_size = os.path.getsize('./../../samplerCensoARG/data/censo_samples/table_f'+str(frac)+'_'+yr+'_ARG.csv')
    np.random.seed(file_size)

    ## Aplicar IDs
    # Generate a unique random number for each row in the DataFrame
    n_digits = 8; n_rows = len(table)
    random_numbers = np.random.randint(10**(n_digits - 1), 10**n_digits, n_rows)
    # Extract the last two digits of the year
    last_two_digits_year = table['ANO4'].apply(lambda x: int(str(x)[-2:]))
    
    # unique ID
    table.insert(0, 'ID', random_numbers * 100 + last_two_digits_year)

    ## Agregar ranking de Region y Aglo
    print(table.shape)
    table = table.merge(AGLO_rk[['AGLOMERADO', 'ANO4', 'AGLO_rk']]).merge(Reg_rk[['Region', 'ANO4', 'Reg_rk']])
    print(table.shape)
    
    table.to_csv('/media/matias/Elements/suite/poblaciones/table_f'+str(frac)+'_'+yr+'_ARG.csv', index = False)  # Copias en carpeta yr_samples, en nuestra carpeta de indice de pobreza
    

2015
(423332, 39)
(423332, 41)
2016
(428659, 39)
(428659, 41)
2017
(434438, 39)
(434438, 41)
2018
(438092, 39)
(438092, 41)
2019
(444062, 39)
(444062, 41)
2020
(447885, 39)
(447885, 41)
2021
(452673, 39)
(452673, 41)
2022
(454792, 39)
(454792, 41)
2023
(458153, 39)
(0, 41)


In [81]:
table.columns

Index(['ID', 'VIVIENDA_REF_ID', 'RADIO_REF_ID', 'TIPVV', 'V01', 'URP', 'DPTO',
       'PROV', 'AGLOMERADO', 'HOGAR_REF_ID', 'H05', 'H06', 'H07', 'H08', 'H09',
       'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'PROP', 'TOTPERS',
       'PERSONA_REF_ID', 'P01', 'P02', 'P03', 'P05', 'P06', 'P07', 'P12',
       'P08', 'P09', 'P10', 'CONDACT', 'IX_TOT', 'AGLO_rk', 'Region', 'ANO4',
       'Reg_rk'],
      dtype='object')